In [4]:
pip install selenium webdriver-manager


In [5]:
# Chrome 설치
!apt-get update
!apt-get install -y wget unzip
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb || apt-get -f install -y

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 http://security.ubuntu.com/ubuntu jammy-security InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

# 크롬 옵션 설정
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.binary_location = "/usr/bin/google-chrome"

# 드라이버 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# URL 접속
url = "https://www.koreabaseball.com/Game/LiveText.aspx?leagueId=1&seriesId=0&gameId=20250322NCHT0&gyear=2025"
driver.get(url)

# 로딩 대기
time.sleep(3)

# 라인업 HTML 추출
element_away = driver.find_element(By.ID, "lineup1")
html_away = element_away.get_attribute("innerHTML")

# BeautifulSoup으로 파싱
soup = BeautifulSoup(html_away, 'html.parser')

# 테이블 구조 파악 후 데이터 추출
rows = soup.find_all('tr')
data_away = []
for row in rows:
    cols = [col.get_text(strip=True) for col in row.find_all(['td', 'th'])]
    if cols:
        data_away.append(cols)

# DataFrame으로 변환
df_away = pd.DataFrame(data_away)

# 컬럼명이 있다면 첫 행을 헤더로 지정
if df_away.shape[0] > 1 and all('타순' in cell or '포지션' in cell for cell in df_away.iloc[0]):
    df_away.columns = df_away.iloc[0]
    df_away = df_away.drop(index=0).reset_index(drop=True)


# 라인업 HTML 추출
element_home = driver.find_element(By.ID, "lineup2")
html_home = element_home.get_attribute("innerHTML")

# BeautifulSoup으로 파싱
soup = BeautifulSoup(html_home, 'html.parser')

# 테이블 구조 파악 후 데이터 추출
rows = soup.find_all('tr')
data_home = []
for row in rows:
    cols = [col.get_text(strip=True) for col in row.find_all(['td', 'th'])]
    if cols:
        data_home.append(cols)

# DataFrame으로 변환
df_home = pd.DataFrame(data_home)

# 컬럼명이 있다면 첫 행을 헤더로 지정
if df_home.shape[0] > 1 and all('타순' in cell or '포지션' in cell for cell in df_home.iloc[0]):
    df_home.columns = df_home.iloc[0]
    df_home = df_home.drop(index=0).reset_index(drop=True)

# 결과 출력
print("[라인업 데이터프레임]")
display(df_away)

display(df_home)

# 종료
driver.quit()


[라인업 데이터프레임]


,0,1,2,3,4,5,6,7,8,9,10,11
0,타자,타수,득점,안타,홈런,타점,도루,희타,볼넷,삼진,병살,실책
1,박민우,5,0,1,0,0,0,0,0,1,0,0
2,김주원,5,1,2,0,0,0,0,0,2,0,0
3,손아섭,3,1,1,0,0,0,0,1,0,0,0
4,김성욱,0,0,0,0,0,0,0,0,0,0,0
5,데이비슨,3,0,0,0,0,0,0,1,1,0,0
6,박건우,4,0,1,0,2,0,0,0,0,0,0
7,권희동,2,0,0,0,0,0,0,1,1,0,0
8,도태훈,2,0,0,0,0,0,0,0,0,0,0
9,천재환,1,0,0,0,0,0,0,0,1,0,0


,0,1,2,3,4,5,6,7,8,9,10,11
0,타자,타수,득점,안타,홈런,타점,도루,희타,볼넷,삼진,병살,실책
1,박찬호,4,1,0,0,0,0,0,1,0,0,0
2,김도영,2,0,1,0,0,0,0,0,0,0,0
3,윤도현,1,0,0,0,0,0,0,0,0,0,0
4,홍종표,2,0,0,0,0,0,0,0,0,0,0
5,나성범,4,1,1,0,1,0,0,0,2,1,0
6,황동하,0,0,0,0,0,0,0,0,0,0,0
7,위즈덤,3,1,0,0,0,0,0,1,1,0,0
8,최형우,4,0,1,0,2,0,0,0,1,0,0
9,박재현,0,1,0,0,0,0,0,0,0,0,0


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

# 셀레니움 설정
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 날짜 범위
dates = pd.date_range("2025-03-22", "2025-08-03")

# 팀별 라인업 저장용
team_lineups = {}

# 경기 목록 가져오기 (샘플: 수동 or 일정 페이지 크롤링해야 자동화 가능)
game_ids = [
    ("2025-03-22", "NCHT0", "NC", "한화"),
    ("2025-03-22", "KTSS0", "KT", "SSG"),
    # 실제로는 모든 날짜/경기에 대해 리스트가 필요
]

def get_lineup(game_id, game_date):
    url = f"https://www.koreabaseball.com/Game/LiveText.aspx?leagueId=1&seriesId=0&gameId={game_id}&gyear=2025"
    driver.get(url)
    time.sleep(2)

    lineups = {}
    for team_num, lineup_id in zip([1, 2], ['lineup1', 'lineup2']):
        try:
            element = driver.find_element(By.ID, lineup_id)
            html = element.get_attribute("innerHTML")
            soup = BeautifulSoup(html, 'html.parser')
            rows = soup.find_all('tr')
            data = []
            for row in rows:
                cols = [col.get_text(strip=True) for col in row.find_all(['td', 'th'])]
                if cols:
                    data.append(cols)
            df = pd.DataFrame(data)
            if df.shape[0] > 1 and all('타순' in cell or '포지션' in cell for cell in df.iloc[0]):
                df.columns = df.iloc[0]
                df = df.drop(index=0).reset_index(drop=True)
            lineups[team_num] = df
        except:
            continue
    return lineups

# 반복 처리
for game_date, game_suffix, away, home in game_ids:
    game_id = game_date.replace("-", "") + game_suffix
    lineups = get_lineup(game_id, game_date)

    for team_id, team_name in zip([1, 2], [away, home]):
        df = lineups.get(team_id)
        if df is None or df.empty:
            continue
        df['날짜'] = game_date
        df['상대팀'] = home if team_name == away else away
        team_lineups.setdefault(team_name, []).append(df)

# 각 팀별 데이터프레임 만들기
team_dfs = {team: pd.concat(dfs, ignore_index=True) for team, dfs in team_lineups.items()}

# 출력 또는 저장
for team, df in team_dfs.items():
    print(f"[{team} 라인업]")
    display(df)
    # df.to_csv(f"{team}_lineup_2025.csv", index=False)

# 종료
driver.quit()


KeyboardInterrupt: 

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import os
from datetime import datetime

# ────────────────────────────────────────────────
# 1. 크롬 드라이버 설정
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.binary_location = "/usr/bin/google-chrome"

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# ────────────────────────────────────────────────
# 2. 경기 일정 CSV 불러오기
df = pd.read_csv("/content/drive/MyDrive/야구 경기 일정.csv")

# 결과 저장 디렉토리 생성
os.makedirs("라인업결과", exist_ok=True)

# ────────────────────────────────────────────────
# 3. 라인업 추출 함수
def extract_lineup(element_id):
    try:
        # 요소가 나타날 때까지 최대 10초 기다림
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, element_id))
        )
        element = driver.find_element(By.ID, element_id)
        html = element.get_attribute("innerHTML")
        soup = BeautifulSoup(html, 'html.parser')
        rows = soup.find_all('tr')
        data = []
        for row in rows:
            cols = [col.get_text(strip=True) for col in row.find_all(['td', 'th'])]
            if cols:
                data.append(cols)
        df = pd.DataFrame(data)
        if df.shape[0] > 1 and all('타순' in cell or '포지션' in cell for cell in df.iloc[0]):
            df.columns = df.iloc[0]
            df = df.drop(index=0).reset_index(drop=True)
        return df
    except Exception as e:
        print(f"[❌ ERROR] {element_id} 요소를 찾을 수 없습니다. → {e}")
        return pd.DataFrame()

# ────────────────────────────────────────────────
# 4. 경기별 라인업 수집 (중복 경기 대응)
df['경기순서'] = df.groupby(['일시', 'AWAY', 'HOME']).cumcount()  # 같은 날 같은 조합이면 순번 부여

for _, row in df.iterrows():
    date_str = row['일시']  # 예: 2025-03-22
    away = row['AWAY'].strip()
    home = row['HOME'].strip()
    order = row['경기순서']  # 0부터 시작

    # 날짜 형식 변환
    dt = datetime.strptime(date_str, "%Y-%m-%d")
    game_id = f"{dt.strftime('%Y%m%d')}{away}{home}{order}"

    # URL 생성
    url = f"https://www.koreabaseball.com/Game/LiveText.aspx?leagueId=1&seriesId=0&gameId={game_id}&gyear={dt.year}"
    driver.get(url)
    time.sleep(2)

    # 라인업 추출
    df_away = extract_lineup("lineup1")
    df_home = extract_lineup("lineup2")

    # 저장
    df_away.to_csv(f"라인업결과/{game_id}_{away}_away.csv", index=False, encoding="utf-8-sig")
    df_home.to_csv(f"라인업결과/{game_id}_{home}_home.csv", index=False, encoding="utf-8-sig")

    print(f"✅ 저장 완료: {game_id}")

# ────────────────────────────────────────────────
# 5. 드라이버 종료
driver.quit()


✅ 저장 완료: 20250322LTLG0
✅ 저장 완료: 20250322OBSK0
✅ 저장 완료: 20250322WOSS0
✅ 저장 완료: 20250322HHKT0
✅ 저장 완료: 20250322NCHT0
✅ 저장 완료: 20250323LTLG0
✅ 저장 완료: 20250323OBSK0
✅ 저장 완료: 20250323WOSS0
✅ 저장 완료: 20250323HHKT0
✅ 저장 완료: 20250323NCHT0
✅ 저장 완료: 20250325HHLG0
✅ 저장 완료: 20250325LTSK0
✅ 저장 완료: 20250325NCSS0
✅ 저장 완료: 20250325OBKT0
✅ 저장 완료: 20250325WOHT0
✅ 저장 완료: 20250326HHLG0
✅ 저장 완료: 20250326LTSK0
✅ 저장 완료: 20250326NCSS0
✅ 저장 완료: 20250326OBKT0
✅ 저장 완료: 20250326WOHT0
✅ 저장 완료: 20250327HHLG0
✅ 저장 완료: 20250327LTSK0
✅ 저장 완료: 20250327NCSS0
✅ 저장 완료: 20250327OBKT0
✅ 저장 완료: 20250327WOHT0
✅ 저장 완료: 20250328SSOB0
✅ 저장 완료: 20250328KTLT0
✅ 저장 완료: 20250328LGNC0
✅ 저장 완료: 20250328SKWO0
✅ 저장 완료: 20250328HTHH0
✅ 저장 완료: 20250329SSOB0
✅ 저장 완료: 20250329KTLT0
✅ 저장 완료: 20250329LGNC0
✅ 저장 완료: 20250329SKWO0
✅ 저장 완료: 20250329HTHH0
✅ 저장 완료: 20250330SSOB0
✅ 저장 완료: 20250330KTLT0
[❌ ERROR] lineup1 요소를 찾을 수 없습니다. → Message: 
Stacktrace:
#0 0x5b2e95ae005a <unknown>
#1 0x5b2e9557fa70 <unknown>
#2 0x5b2e955d1907 <unknown>
#3 0x5b2

In [ ]:
import shutil
shutil.make_archive('/content/라인업결과', 'zip', '/content/라인업결과')  # zip 파일 생성

from google.colab import files
files.download('/content/라인업결과.zip')  # 압축된 파일 다운로드


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>